In [12]:
from docarray import Document, DocumentArray
from jina import Flow
import json
da = DocumentArray()
  
with open('/home/aswin/data/icecat-products-w_price-19k-20201127/icecat-products-w_price-19k-20201127.json') as f:
    jdocs = json.load(f)
for doc in jdocs:
    d = Document(text=" ".join([doc[item] for item in doc if item.startswith("title") 
    or item.startswith("attr_t")
    or item.startswith("short_description") 
    or item.startswith("name")
    or item.startswith("supplier")]), tags=doc)
    da.append(d)

da.summary()
da[0]


╭──────────────── Documents Summary ────────────────╮
│                                                   │
│   Type                   DocumentArrayInMemory    │
│   Length                 19406                    │
│   Homogenous Documents   True                     │
│   Common Attributes      ('id', 'text', 'tags')   │
│   Multimodal dataclass   False                    │
│                                                   │
╰───────────────────────────────────────────────────╯
╭───────────────────── Attributes Summary ─────────────────────╮
│                                                              │
│   Attribute   Data type   #Unique values   Has empty value   │
│  ──────────────────────────────────────────────────────────  │
│   id          ('str',)    19406            False             │
│   tags        ('dict',)   19406            False             │
│   text        ('str',)    19304            False             │
│                                                              │
╰──────────────────────────────────────────────────────────────╯

📄 Document: 81df7fe404106079970b55f08eceb8b0
╭───────────┬──────────────────────────────────────────────────────────────────╮
│ Attribute │ Value                                                            │
├───────────┼──────────────────────────────────────────────────────────────────┤
│ text      │ 006R90321 Xerox 006R90321 toner cartridge Original Black 6 pc(s) │
│           │ Toner (6 Per Box) for CopyCentre C6... (length: 167)             │
│ tags      │ {'id': '3920564', 'name': '006R90321', 'title': 'Xerox 006R90321 │
│           │ toner cartridge Original Black 6 pc(s)', 'ean':                  │
│           │ ['0095205603217'], 'short_description': 'Toner (6 Per Box) for   │
│           │ CopyCentre C65 Digital Copier', 'img_high':                      │
│           │ 'http://images.icecat.biz/img/gallery/img_3920564_high_14726187… │
│           │ 'img_low':                                                       │
│           │ 'http://images.icecat.biz/img/gallery_lows/img_392

In [15]:
f.save_config("flow_aut.yml")

In [13]:
da = da[:111]
from jina import Flow
f = Flow()

f = Flow().add(name='encoder', uses='jinahub://TransformerTorchEncoder/latest', install_requirements=True).add(uses='jinahub://AnnLiteIndexer/latest', install_requirements=True, uses_with={'columns': [('supplier', 'str'), ('price', 'float'), ('attr_t_product_type', 'str'), ('attr_t_product_colour', 'str')], 'n_dim': 768})

with f:
  f.index(da)

────────────────────────── 🎉 Flow is ready to serve! ──────────────────────────
╭────────────── 🔗 Endpoint ───────────────╮
│  ⛓     Protocol                    GRPC  │
│  🏠       Local           0.0.0.0:60079  │
│  🔒     Private    192.168.155.39:60079  │
│  🌍      Public    87.191.159.105:60079  │
╰──────────────────────────────────────────╯



ERROR  GRPCClient@53977 gRPC error: StatusCode.UNAVAILABLE   [09/28/22 14:02:54]
       keepalive watchdog timeout                                               
       The ongoing request is terminated as the server is                       
       not available or closed already.                                         


Exception in thread Thread-137:
Traceback (most recent call last):
  File "/home/aswin/anaconda3/envs/jina/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/home/aswin/anaconda3/envs/jina/lib/python3.9/site-packages/jina/helper.py", line 1299, in run
    self.result = asyncio.run(func(*args, **kwargs))
  File "/home/aswin/anaconda3/envs/jina/lib/python3.9/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/home/aswin/anaconda3/envs/jina/lib/python3.9/asyncio/base_events.py", line 647, in run_until_complete
    return future.result()
  File "/home/aswin/anaconda3/envs/jina/lib/python3.9/site-packages/jina/clients/mixin.py", line 256, in _get_results
    async for resp in c._get_results(*args, **kwargs):
  File "/home/aswin/anaconda3/envs/jina/lib/python3.9/site-packages/jina/clients/base/grpc.py", line 119, in _get_results
    raise ConnectionError(my_details) from None
ConnectionError: keepalive watchdog timeout


ERROR  encoder/rep-0@53977 Exception('Shutdown signal was    [09/28/22 14:03:55]
       not received for 60 seconds') during <bound method                       
       BasePod.close of <jina.orchestrate.pods.Pod object at                    
       0x7feebca868e0>>                                                         
        add "--quiet-error" to suppress the exception                           
       details                                                                  
       ╭──────── Traceback (most recent call last) ────────╮                    
       │ /home/aswin/anaconda3/envs/jina/lib/python3.9/si… │                    
       │ in run_async                                      │                    
       │                                                   │                    
       │   1311 │   │   │   thread.start()                 │                    
       │   1312 │   │   │   thread.join()                  │                    
       │   1313 │   │   │   

BadClient: something wrong when running the eventloop, result can not be retrieved

In [6]:
#define the filter
colors = ["Black", "Blue"]

filter = {
    "attr_t_product_colour": {
        "$in": colors
    }
}

text_string = "laptop case"
text_query = Document(text=text_string)

In [7]:
#apply the filter
with f:
  text_matches = f.search(text_query, show_progress=True, parameters={"filter": filter})

────────────────────────── 🎉 Flow is ready to serve! ──────────────────────────
╭────────────── 🔗 Endpoint ───────────────╮
│  ⛓     Protocol                    GRPC  │
│  🏠       Local           0.0.0.0:62504  │
│  🔒     Private    192.168.155.39:62504  │
│  🌍      Public    87.191.159.105:62504  │
╰──────────────────────────────────────────╯



Output()

In [8]:
text_matches[0].matches[0]

📄 Document: bd7b37315dafc44a1601d211d3c22f45
╭───────────┬──────────────────────────────────────────────────────────────────╮
│ Attribute │ Value                                                            │
├───────────┼──────────────────────────────────────────────────────────────────┤
│ adjacency │ 1                                                                │
│ text      │ ProBook 4320s Notebook PC HP ProBook 4320s Notebook PC Black     │
│           │ 33.8 cm (13.3") 1366 x 768 pixels 3rd g... (length: 1950)        │
│ tags      │ {'attr_t_battery_type': '6-cell (47 WHr) Lithium-Ion Battery',   │
│           │ 'attr_t_display': 'LCD',                                         │
│           │ 'attr_b_intel__clear_video_hd_technology__intel__cvt_hd_': True, │
│           │ 'attr_b_fsb_parity': False, 'attr_b_tv_in_port': False,          │
│           │ 'attr_t_product_colour': 'Black', 'attr_t_cable_lock_slot_type': │
│           │ 'Kensington', 'attr_n_number_of_built_in_speakers'

In [24]:
query = Document(text='Cartridge')

with f:
  results = f.search(query)
  print(results[0].match.text)


Output()

─────────────────────────────────────────── 🎉 Flow is ready to serve! ────────────────────────────────────────────
╭────────────── 🔗 Endpoint ───────────────╮
│  ⛓     Protocol                    GRPC  │
│  🏠       Local           ]8;id=935852;grpc://0.0.0.0:64531\0.0.0.0]8;;\]8;id=977634;grpc://0.0.0.0:64531\:]8;;\]8;id=471392;grpc://0.0.0.0:64531\64531]8;;\  │
│  🔒     Private    ]8;id=33399;grpc://192.168.155.39:64531\192.168.155.39]8;;\]8;id=979289;grpc://192.168.155.39:64531\:]8;;\]8;id=266691;grpc://192.168.155.39:64531\64531]8;;\  │
│  🌍      Public    ]8;id=337967;grpc://87.191.159.105:64531\87.191.159.105]8;;\]8;id=742169;grpc://87.191.159.105:64531\:]8;;\]8;id=692064;grpc://87.191.159.105:64531\64531]8;;\  │
╰──────────────────────────────────────────╯

AttributeError: 'function' object has no attribute 'text'

In [8]:
from jina import Flowb
from docarray import Document, DocumentArray

f = Flow().add(uses='jinahub://TransformerTorchEncoder/latest').add(uses='jinahub://SimpleIndexer')

with f:
  r = f.post('/', inputs=DocumentArray([Document(text='hello')]))
  print(r.to_json())

UserWarning: VersionConflict(torch 1.12.0+cu113 (/home/aswin/anaconda3/envs/jina/lib/python3.9/site-packages), 
Requirement.parse('torch==1.9.0+cpu')) (raised from 
/home/aswin/anaconda3/envs/jina/lib/python3.9/site-packages/jina/hubble/helper.py:419)

Output()

─────────────────────────────────────────── 🎉 Flow is ready to serve! ────────────────────────────────────────────
╭────────────── 🔗 Endpoint ───────────────╮
│  ⛓     Protocol                    GRPC  │
│  🏠       Local           ]8;id=188688;grpc://0.0.0.0:53671\0.0.0.0]8;;\]8;id=698765;grpc://0.0.0.0:53671\:]8;;\]8;id=824615;grpc://0.0.0.0:53671\53671]8;;\  │
│  🔒     Private    ]8;id=101345;grpc://192.168.155.39:53671\192.168.155.39]8;;\]8;id=156677;grpc://192.168.155.39:53671\:]8;;\]8;id=845611;grpc://192.168.155.39:53671\53671]8;;\  │
│  🌍      Public    ]8;id=575784;grpc://87.191.159.105:53671\87.191.159.105]8;;\]8;id=226118;grpc://87.191.159.105:53671\:]8;;\]8;id=204543;grpc://87.191.159.105:53671\53671]8;;\  │
╰──────────────────────────────────────────╯

[{"id": "c655b7110083fa6be3991e58ff92198d", "parent_id": null, "granularity": null, "adjacency": null, "blob": null, "tensor": null, "mime_type": null, "text": "hello", "weight": null, "uri": null, "tags": null, "offset": null, "location": null, "embedding": [0.08156412839889526, -0.0165849681943655, -0.005647609476000071, 0.03694671764969826, 0.07050866633653641, 0.01082331221550703, -0.010305303148925304, 0.09388928860425949, 0.03058224357664585, -0.010133092291653156, 0.035235609859228134, -0.03164002299308777, -0.000426911428803578, 0.057421792298555374, 0.08025908470153809, -0.28108707070350647, 0.09428101778030396, 0.017559310421347618, -0.14722050726413727, -0.0006037379498593509, -0.020847491919994354, -0.012616757303476334, -5.8429937780601904e-05, 0.09862927347421646, 0.05970126390457153, 0.01143552828580141, -0.0028616294730454683, 0.0197420883923769, 0.0310052502900362, 0.05060402676463127, -0.03180179372429848, 0.005787048488855362, 0.07171972841024399, 0.04280797019600868

In [25]:
r[0].matches

╭────────── Documents Summary ──────────╮
│                                       │
│   Type                   MatchArray   │
│   Length                 0            │
│   Homogenous Documents   False        │
│   Multimodal dataclass   True         │
│                                       │
╰───────────────────────────────────────╯